In [8]:
import matplotlib.pyplot as plt
import numpy as np
import random
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import argparse
import numpy as np
import time
from copy import deepcopy
import seaborn as sns 
import matplotlib.pyplot as plt

In [9]:
import numpy as np
private_labels=np.load("Nohot_private_label.npy")
private_pixels=np.load("private_pixels.npy")
public_labels=np.load("Nohot_public_label.npy")
public_pixels=np.load("public_pixels.npy")
train_labels=np.load("Nohot_train_label.npy")
train_pixels=np.load("train_pixels.npy")

In [10]:
X_train , y_train = train_pixels,train_labels
X_valid , y_valid = public_pixels,public_labels
X_test , y_test = private_pixels,private_labels

In [11]:
mean_vals = np.mean(X_train, axis=0)
std_val = np.std(X_train)

X_train_centered = (X_train - mean_vals)/std_val
X_valid_centered = (X_valid - mean_vals)/std_val
X_test_centered = (X_test - mean_vals)/std_val

In [12]:
import torch
import torch.nn
input_data=torch.FloatTensor(X_train_centered)
label=torch.LongTensor(y_train)

In [13]:
input_data1=torch.FloatTensor(X_valid_centered)
label1=torch.LongTensor(y_valid)

In [14]:
input_data2=torch.FloatTensor(X_test_centered)
label2=torch.LongTensor(y_test)

In [15]:
input_data=input_data.transpose(1,3)
input_data1=input_data1.transpose(1,3)
input_data2=input_data2.transpose(1,3)

In [16]:
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
trainset = TensorDataset(input_data,label)
valset = TensorDataset(input_data1,label1)
testset = TensorDataset(input_data2,label2)

In [17]:
partition={'train':trainset,'val':valset,'test':testset}

In [18]:
import torch.nn as nn
import torch

In [19]:
cfg = {
    'VGG11': [64, 'M', 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG13': [64, 64, 'M', 128, 128, 'M', 256, 256, 'M', 512, 512, 'M', 512, 512, 'M'],
    'VGG16': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 'M', 512, 512, 512, 'M', 512, 512, 512, 'M'],
    'VGG19': [64, 64, 'M', 128, 128, 'M', 256, 256, 256, 256, 'M', 512, 512, 512, 512, 'M', 512, 512, 512, 512, 'M'],
}

class VGG(nn.Module):
    def __init__(self,vgg_name,num_classes=7):
        super(VGG,self).__init__()
        self.features= self._make_layers(cfg[vgg_name]) # 함수 앞 _는 private
        self.classifier = nn.Linear(512, num_classes)
        
    def forward(self,x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    def _make_layers(self,cfg):
        layers=[]
        in_channels = 1
        for x in cfg:
            if x == 'M':
                layers += [nn.MaxPool2d(kernel_size=2,stride=2)]
            else :
                layers +=[nn.Conv2d(in_channels, x, kernel_size=3, padding=1),
                          nn.BatchNorm2d(x),
                          nn.ReLU(inplace=True)]
                in_channels =x
        layers+=[nn.AvgPool2d(kernel_size=1,stride=1)]
        return nn.Sequential(*layers)
        

In [20]:
def test():
    net = VGG('VGG11')
    x = torch.randn(2,1,48,48)
    y = net(x)
    print(y.size())

In [21]:
test()

torch.Size([2, 7])


In [22]:
def train(net,partition,optimizer,criterion):
    trainloader = torch.utils.data.DataLoader(partition['train'],
                                             batch_size=2048,
                                             shuffle=True)
    net.train()
    correct = 0
    total =0
    train_loss = 0.0
    for i, data in enumerate(tqdm(trainloader, 0)):
        time.sleep(0.0000001)
        optimizer.zero_grad()
        inputs, labels = data
        inputs = inputs.cuda()
        labels = labels.cuda()
        outputs = net(inputs)
        #break
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        train_loss += loss.item()
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
        #if i % 200 == 0:
        #    print(correct,train_loss)
    train_loss = train_loss / len(trainloader)
    train_acc = 100 * correct / total
    return net, train_loss, train_acc

In [23]:
def test(net, partition):
    testloader = torch.utils.data.DataLoader(partition['test'], 
                                             batch_size=2048, 
                                             shuffle=False, num_workers=2)
    net.eval()
    
    correct = 0
    total = 0
    with torch.no_grad():
        for data in tqdm(testloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        test_acc = 100 * correct / total
    return test_acc

In [24]:
def validate(net, partition, criterion):
    valloader = torch.utils.data.DataLoader(partition['val'], 
                                            batch_size=2048, 
                                            shuffle=False, num_workers=2)
    net.eval()

    correct = 0
    total = 0
    val_loss = 0 
    with torch.no_grad():
        for data in tqdm(valloader):
            time.sleep(0.0000001)
            images, labels = data
            images = images.cuda()
            labels = labels.cuda()
            outputs = net(images)

            loss = criterion(outputs, labels)
            
            val_loss += loss.item()
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

        val_loss = val_loss / len(valloader)
        val_acc = 100 * correct / total
    return val_loss, val_acc

In [25]:
from tqdm import tqdm, tqdm_notebook
from tqdm import trange
import time

In [27]:
seed = 123
np.random.seed(seed)
torch.manual_seed(seed)
net=VGG('VGG11')
net.cuda()
criterion=nn.CrossEntropyLoss()
optimizer = optim.Adam(net.parameters(), lr=0.0015, weight_decay=0.00001)
train_losses=[]
val_losses=[]
train_accs=[]
val_accs=[]
for epoch in range(100):
    ts=time.time()
    net, train_loss,train_acc=train(net, partition, optimizer, criterion)
    val_loss, val_acc = validate(net, partition, criterion)
    te=time.time()
    print('time : {:2.2f}, train_loss : {:2.2f}, train_acc : {:2.2f},val_acc : {:2.2f},val_loss : {:2.2f}'.format(te-ts,train_loss,train_acc,val_acc,val_loss))
    train_loss=round(train_loss,2)
    val_loss=round(val_loss,2)
    train_acc=round(train_acc,2)
    val_acc=round(val_acc,2)
    train_losses.append(train_loss)
    val_losses.append(val_loss)
    train_accs.append(train_acc)
    val_accs.append(val_acc)
test_acc = test(net,partition)
test_acc=round(test_acc,2)

  0%|          | 0/15 [00:00<?, ?it/s]

time : 7.51, train_loss : 2.13, train_acc : 20.19,val_acc : 20.45,val_loss : 4.21


  0%|          | 0/15 [00:00<?, ?it/s]

time : 6.99, train_loss : 1.82, train_acc : 24.69,val_acc : 24.27,val_loss : 1.81


  0%|          | 0/15 [00:00<?, ?it/s]

...

  0%|          | 0/15 [00:00<?, ?it/s]

time : 7.34, train_loss : 0.01, train_acc : 99.68,val_acc : 60.66,val_loss : 2.49


  0%|          | 0/2 [00:00<?, ?it/s]

time : 7.32, train_loss : 0.02, train_acc : 99.61,val_acc : 59.43,val_loss : 2.68


100%|██████████| 2/2 [00:00<00:00,  3.02it/s]


In [28]:
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/VGG_train_loss.npy",train_loss)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/VGG_val_loss.npy",val_loss)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/VGG_train_acc.npy",train_acc)
np.save("/home/lab/양창희/Emotion Project/fer2013/result_CNN_pytorch/VGG_val_acc.npy",val_acc)